In [2]:
# Imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from edinburgh_challenge.constants import police_stations

In [3]:
source = "./data.xlsx"

In [4]:
data = pd.read_excel(source)
data["Time"] = (data["Day"]-1)*24 + data["Hour"]
data.columns = [x.lower() for x in data.columns]

In [4]:
data

,urn,latitude,longitude,day,hour,priority,deployment time (hrs),time
0,PS-20220706-0009,55.782705,-4.432785,1,0,Prompt,0.60,0
1,PS-20220706-0021,55.849990,-4.095880,1,0,Prompt,1.55,0
2,PS-20220706-0028,55.849864,-4.307251,1,0,Prompt,1.10,0
3,PS-20220706-0035,55.869998,-4.096223,1,0,Prompt,0.85,0
4,PS-20220706-0043,55.810980,-4.303656,1,0,Prompt,2.50,0
...,...,...,...,...,...,...,...,...
2267,PS-20220715-3261,55.865248,-4.283326,7,23,Prompt,0.75,167
2268,PS-20220715-3270,55.897534,-4.367379,7,23,Immediate,1.25,167
2269,PS-20220715-3276,55.892101,-4.326152,7,23,Prompt,0.50,167
2270,PS-20220715-3279,55.882194,-4.327238,7,23,Immediate,1.35,167


In [5]:
class NaiveModel():
    def make_allocation(self, incidents, officers, current_time):
        # Sort incidents by priority
        incidents.sort(key=lambda inc: inc.priority)

        allocations = {}
        officers_allocated = []
        for inc in incidents:
            allocated = False
            # Sort stations by distance to the incident
            sorted_stations = sorted(inc.distances, key=inc.distances.get)

            for station in sorted_stations:
                # Check for available officer in the station
                available_officers = [off for off in officers[station] if (off.available and off not in officers_allocated) ]
                if available_officers:
                    # Allocate the first available officer
                    chosen_officer = available_officers[0]
                    allocations[inc.urn] = chosen_officer.name
                    officers_allocated.append(chosen_officer)
                    allocated = True
                    break
            
            if not allocated:
                # No officers available for this incident
                allocations[inc.urn] = None

        return allocations


In [6]:
from dataclasses import dataclass
from typing import List, Dict
import numpy as np
from abc import ABC, abstractmethod

class Model(ABC):
    @abstractmethod
    def make_allocation(self, incidents, officers, current_time):
        pass

@dataclass
class Incident:
    urn: str
    latitude: float
    longitude: float
    day:int
    hour:int
    global_time: float  # Time at which the incident was reported (24*day + hour)
    deployment_time: float  # Time taken to resolve the incident after reaching the location
    priority: str
    distances: Dict[str, float]  # Distances from each station
    resolved: bool = False
    resolving_officer: str = None
    response_time: float = None  # Global time when the response arrives at the scene
    resolution_time: float = None  # Global time when the incident is resolved

@dataclass
class Officer:
    name: str
    station: str
    available: bool = True
    return_time: float = 0.0
        
from typing import Dict, List

# Shift distribution structure
# Example: {'Early': {'Station_1': 5, 'Station_2': 5, 'Station_3': 5}, ...}
ShiftDistribution = Dict[str, Dict[str, int]]

In [10]:
from math import radians, cos, sin, asin, sqrt

class Simulation:
    SPEED_MPH = 30  # Speed in miles per hour

    def __init__(self, df: pd.DataFrame, station_coords: List[tuple], shift_distribution: ShiftDistribution, verbose:int=0):
        self.df = df.copy()
        self.station_coords = station_coords
        self.shift_distribution = shift_distribution
        self.current_time = 0
        self.officers = { key: [] for key in shift_distribution["Early"].keys() } # Station_1, Station_2 and Station_3
        self.verbose = verbose

        # This resolved incidents is made to keep a track of when an incident
        # is allocated to an officer. This is to conduct the evaluating model check
        self.resolved_incidents = [] # This list keeps a track and order of the resolved incidents as they are allocated
    
        
        # Making the df compatible with data structures
        df.columns = [x.lower() for x in df.columns]
        

    def print_dashboard(self, allocations):
        print(f"\n--- Day {self.current_time // 24 + 1}, Hour {self.current_time % 24} ---")
        pending_cases = [inc for inc in self.cumulative_incidents if not inc.resolved]
        print(f"Pending Cases: {len(pending_cases)}")
        print("Pending Incident URNs:", [inc.urn for inc in pending_cases])

        inv_allocations = {v: k for k, v in allocations.items()}
        
        for station, officers in self.officers.items():
            print(f"\n{station}:")
            for officer in officers:
                # Find the incident the officer is currently assigned to
                if officer.name in inv_allocations.keys():
                    status = inv_allocations[officer.name]
                else: 
                    status = "Busy"
                print(f"  - {officer.name}: {status}")
        
    def calculate_distance(self, lat1, lon1, lat2, lon2):
        """
        Calculate the great circle distance between two points 
        on the earth (specified in decimal degrees)
        """
        # Convert decimal degrees to radians 
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

        # Haversine formula 
        dlat = lat2 - lat1 
        dlon = lon2 - lon1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        r = 3956  # Radius of Earth in miles. Use 6371 for kilometers
        return c * r

    def generate_incidents_for_hour(self, global_time):
        hour_incidents = self.df[(self.df['time'] == global_time)]
        incidents = []
        for _, row in hour_incidents.iterrows():
            deployment_time = row.pop("deployment time (hrs)")
            row.pop("time")  # Not needed as we use global_time
            distances = {f'Station_{i+1}': self.calculate_distance(row['latitude'], row['longitude'], lat, lon) 
                         for i, (lat, lon) in enumerate(self.station_coords)}
            incidents.append(Incident(**row, 
                                      deployment_time=deployment_time, 
                                      distances=distances,
                                      global_time=global_time))
        return incidents


    def update_officers_for_shift(self, shift):
        for station, num_officers in self.shift_distribution[shift].items():
            # Update the number of officers for each station
            self.officers[station] = [Officer(f"Officer_{station}_{shift}_{i}", station=station) for i in range(num_officers)]

    def process_allocations(self, allocations):
        if self.verbose > 0:
            print(f"{allocations=}")
        for urn, officer_id in allocations.items():
            incident = next((inc for inc in self.cumulative_incidents if inc.urn == urn), None)
            #print("incident:", incident)
            if officer_id is None:
                if incident:
                    incident.resolved = False
                continue

            officer = next((off for station_officers in self.officers.values() for off in station_officers if off.name == officer_id), None)
            if incident and officer:
                travel_time = incident.distances[officer.station] / self.SPEED_MPH
                officer.available = False  # Mark officer as busy
                officer.return_time = self.current_time + travel_time + incident.deployment_time
                
                incident.resolved = True  # Mark incident as resolved
                incident.resolving_officer = officer.name  # Assign officer to incident
                incident.response_time = self.current_time + travel_time # Global time when the response reached
                incident.resolution_time = officer.return_time # Global Time when the incident was resolved
                self.resolved_incidents.append(incident)

                
    def update_officer_availability(self):
        for station, officers in self.officers.items():
            for officer in officers:
                if not officer.available and self.current_time >= officer.return_time:
                    officer.available = True


    def run(self, model):
        self.cumulative_incidents = []  # Global list to track all incidents

        while self.current_time < 24 * 7:  # For a week-long simulation
            
            day = self.current_time // 24 + 1
            hour = self.current_time % 24
            
            # Update officer availability at the start of each timestep
            self.update_officer_availability()

            # Update officers for shift change
            if hour in [0, 8, 16]:
                shift = 'Early' if hour == 0 else 'Day' if hour == 8 else 'Night'
                #print(f"{shift=}")
                self.update_officers_for_shift(shift)
            
            total_officers = len(self.officers["Station_1"]) + len(self.officers["Station_2"]) + len(self.officers["Station_3"]) 
            #print(f"{total_officers=}")
                
            # Generate and add new incidents
            new_incidents = self.generate_incidents_for_hour(self.current_time)
            self.cumulative_incidents.extend(new_incidents)
            
            # Filter to get only pending incidents
            pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
            #print(f"{len(pending_incidents)=}")
            #print(f"{pending_incidents=}")
            # Call the model to allocate incidents
            #print(f"{self.officers=}")
            allocations = model.make_allocation(pending_incidents, self.officers, self.current_time)
            
            # Process allocations and update the state
            self.process_allocations(allocations)

            if self.verbose > 2:
                self.print_dashboard(allocations)
            
            
            # Pending cases after allocation
            pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
            #print(f"{pending_incidents=}")
            
            self.current_time += 1  # Move to the next hour
            if self.verbose > 3:
                input("Press Enter to continue to the next hour...\n")
            #print()
            #print([inc.urn for inc in self.cumulative_incidents])
            #print("*"*50)

    # Checks and Analysis
    def analyze_simulation_results(self):

        simulation = self
        
        # Initialize counters and accumulators
        incident_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        resolved_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        within_threshold_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        response_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        resolution_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        deployment_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        thresholds = {'Immediate': 1, 'Prompt': 3, 'Standard': 6}
        total_officer_hours = {officer.name: 0 for station in simulation.officers.values() for officer in station}
        unresolved_incidents = 0
    
        # Analyze each incident
        for incident in simulation.cumulative_incidents:
            priority = incident.priority
            incident_counts[priority] += 1
    
            if incident.resolved:
                resolved_counts[priority] += 1
                
                response_time = incident.response_time
                resolution_time = incident.resolution_time
                
                response_times[priority].append(response_time)
                resolution_times[priority].append(resolution_time)
                
                deployment_time = incident.deployment_time
                deployment_times[priority].append(deployment_time)
                
                # Calculate the time from incident report to response arrival
                time_to_response = incident.response_time - incident.global_time
    
    
                # Check if the response was within the threshold
                if time_to_response <= thresholds[priority]:
                    within_threshold_counts[priority] += 1
            else:
                unresolved_incidents += 1
    
        # Calculate percentages and mean times
        completion_percentages = {p: (resolved_counts[p] / incident_counts[p]) * 100 if incident_counts[p] > 0 else 0 for p in incident_counts}
        mean_response_times = {p: np.mean(response_times[p]) if response_times[p] else 0 for p in response_times}
        mean_deployment_times = {p: np.mean(deployment_times[p]) if deployment_times[p] else 0 for p in deployment_times}
        threshold_compliance = {p: (within_threshold_counts[p] / resolved_counts[p]) * 100 if resolved_counts[p] > 0 else 0 for p in incident_counts}
    
        # Calculate officer utilization
        for station in simulation.officers.values():
            for officer in station:
                if not officer.available:
                    total_officer_hours[officer.name] += (simulation.current_time - officer.return_time)
        officer_utilization = sum(total_officer_hours.values()) / (len(total_officer_hours) * simulation.current_time) * 100
    
        unresolved_incident_percentage = (unresolved_incidents / len(simulation.cumulative_incidents)) * 100 if simulation.cumulative_incidents else 0
    
        return {
            "Completion Percentages": completion_percentages,
            "Mean Response Times": mean_response_times,
            "Mean Deployment Times": mean_deployment_times,
            "Threshold Compliance": threshold_compliance,
            "Officer Utilization": officer_utilization,
            "Unresolved Incident Percentage": unresolved_incident_percentage
        }

    def check_simulation(self):
        simulation = self
        # Initialize officer assignments based on all shifts
        officer_assignments = {}
        for shift, stations in shift_distribution.items():
            for station, num_officers in stations.items():
                for i in range(num_officers):
                    officer_name = f"Officer_{station}_{shift}_{i}"
                    officer_assignments[officer_name] = []
        
        incident_response = {'Immediate': {'total': 0, 'within_time': 0}, 
                             'Prompt': {'total': 0, 'within_time': 0}, 
                             'Standard': {'total': 0, 'within_time': 0}}
        
        time_travel_occurred = False
    
        for incident in simulation.resolved_incidents:
            if incident.resolved:
                # Check officer assignments and time traveling
                if incident.resolving_officer:
                    officer_assignments[incident.resolving_officer].append(incident.resolution_time)
                    if len(officer_assignments[incident.resolving_officer]) > 1:
                        if officer_assignments[incident.resolving_officer][-2] > incident.resolution_time:
                            #print(f"{officer_assignments[incident.resolving_officer]=}")
                            #print(f"{incident.resolving_officer=}")
                            time_travel_occurred = True
    
                # Count incidents and check response time
                incident_response[incident.priority]['total'] += 1
                target_time = {'Immediate': 1, 'Prompt': 3, 'Standard': 6}[incident.priority]
                if incident.response_time - incident.global_time <= target_time:
                    incident_response[incident.priority]['within_time'] += 1
    
        # Calculate percentages
        for priority in incident_response:
            total = incident_response[priority]['total']
            if total > 0:
                incident_response[priority]['percentage'] = (incident_response[priority]['within_time'] / total) * 100
            else:
                incident_response[priority]['percentage'] = 0
    
        return officer_assignments, incident_response, time_travel_occurred

In [11]:
shift_distribution = {
    'Early': {'Station_1': 5, 'Station_2': 5, 'Station_3': 5},
    'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
    'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}
}

ps_coords = [ (p.x, p.y) for p in 
                [police_stations.one, 
                 police_stations.two, 
                 police_stations.three]]
num_officers_per_station = [4, 5, 6]
simulation = Simulation(data, ps_coords, shift_distribution, 
                        verbose=0)

In [9]:
naive_model = NaiveModel()

In [10]:
simulation.run(naive_model)

In [11]:
officer_assignments, incident_response, time_travel_occurred = simulation.check_simulation()
if time_travel_occurred:
    print("ERROR: Time Travel Occured")

In [12]:
analysis = simulation.analyze_simulation_results()
analysis

{'Completion Percentages': {'Immediate': 100.0,
  'Prompt': 100.0,
  'Standard': 94.8473282442748},
 'Mean Response Times': {'Immediate': 88.31337990888726,
  'Prompt': 84.94377363616414,
  'Standard': 91.85488437917675},
 'Mean Deployment Times': {'Immediate': 1.5403225806451613,
  'Prompt': 1.5061196105702366,
  'Standard': 1.4886317907444666},
 'Threshold Compliance': {'Immediate': 99.35483870967742,
  'Prompt': 95.54937413073714,
  'Standard': 34.80885311871227},
 'Officer Utilization': -0.021441988491633236,
 'Unresolved Incident Percentage': 1.1883802816901408}

### Simulation Checks and Analysis

Currently, the simulation check says that there is time travelling going on. However, this is a false alarm. The simulation thinks that as we read the events in the order of cumulative list(these are ordered as they are received. Basically, this order should be the same as the order of the dataset). However, beaucase incidents are prioritised in accordance to their priority, there is small chance that an incident with a lower urn but higher priority is done first. The simulation thinks that this is proof of time travelling. However, this isn't how we should be checking for time travelling.

Update: Added a list called resolved_incident to simulation that keeps a track of when something was assigned to a person (essentailly). Hence, it becomes easier to check for time travelling on this list instead of cumulative list. The logic of the check_simulation function was not changed.

### EnhancedModel testing

In [13]:
class EnhancedModel(Model):
    def make_allocation(self, incidents, officers, current_time):
        # Adjusting the priority mechanism to balance between priority, waiting time, and travel time
        incidents.sort(key=lambda inc: (inc.priority, current_time - inc.global_time, min(inc.distances.values())))

        allocations = {}
        allocated_officers = set()  # Set to keep track of officers already allocated

        for inc in incidents:
            # Find the nearest station with available officers
            nearest_stations = sorted(inc.distances, key=inc.distances.get)

            for station in nearest_stations:
                # Filter out officers who are already allocated
                available_officers = [off for off in officers[station] if off.available and off.name not in allocated_officers]

                if available_officers:
                    # Allocate the first available officer
                    chosen_officer = available_officers[0]
                    allocations[inc.urn] = chosen_officer.name
                    allocated_officers.add(chosen_officer.name)  # Mark officer as allocated
                    chosen_officer.available = False  # Mark officer as busy
                    # Assuming return_time is calculated elsewhere
                    break
            else:
                # No officers available for this incident
                allocations[inc.urn] = None

        return allocations


In [14]:
shift_distribution = {
    'Early': {'Station_1': 5, 'Station_2': 5, 'Station_3': 5},
    'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
    'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}
}

ps_coords = [ (p.x, p.y) for p in 
                [police_stations.one, 
                 police_stations.two, 
                 police_stations.three]]
num_officers_per_station = [4, 5, 6]
simulation = Simulation(data, ps_coords, shift_distribution, 
                        verbose=1)

In [121]:
naive_model = NaiveModel()
simulation.run(naive_model)

In [122]:
officer_assignments, incident_response, time_travel_occurred = simulation.check_simulation()
if time_travel_occurred:
    print("ERROR: Time Travel Occured")

In [123]:
analysis = simulation.analyze_simulation_results()
analysis

{'Completion Percentages': {'Immediate': 0, 'Prompt': 0, 'Standard': 0},
 'Mean Response Times': {'Immediate': 0, 'Prompt': 0, 'Standard': 0},
 'Mean Deployment Times': {'Immediate': 0, 'Prompt': 0, 'Standard': 0},
 'Threshold Compliance': {'Immediate': 0, 'Prompt': 0, 'Standard': 0},
 'Total Officer Hours': 0,
 'Unresolved Incident Percentage': 0}

In [18]:
{'Completion Percentages': {'Immediate': 100.0,
  'Prompt': 100.0,
  'Standard': 97.26651480637814},
 'Mean Response Times': {'Immediate': 76.69943973316968,
  'Prompt': 72.79034842356631,
  'Standard': 81.4700081932075},
 'Mean Deployment Times': {'Immediate': 1.5379699248120302,
  'Prompt': 1.5070445344129555,
  'Standard': 1.4734192037470728},
 'Threshold Compliance': {'Immediate': 99.62406015037594,
  'Prompt': 94.8178137651822,
  'Standard': 28.805620608899297},
 'Officer Utilization': -0.11015434113684058,
 'Unresolved Incident Percentage': 0.6185567010309279}

{'Completion Percentages': {'Immediate': 100.0,
  'Prompt': 100.0,
  'Standard': 97.26651480637814},
 'Mean Response Times': {'Immediate': 76.69943973316968,
  'Prompt': 72.79034842356631,
  'Standard': 81.4700081932075},
 'Mean Deployment Times': {'Immediate': 1.5379699248120302,
  'Prompt': 1.5070445344129555,
  'Standard': 1.4734192037470728},
 'Threshold Compliance': {'Immediate': 99.62406015037594,
  'Prompt': 94.8178137651822,
  'Standard': 28.805620608899297},
 'Officer Utilization': -0.11015434113684058,
 'Unresolved Incident Percentage': 0.6185567010309279}

# Fixing the Simulation

In [119]:
from math import radians, cos, sin, asin, sqrt

class Simulation:
    SPEED_MPH = 30  # Speed in miles per hour

    def __init__(self, df: pd.DataFrame, station_coords: List[tuple], shift_distribution: ShiftDistribution, verbose:int=0):
        self.df = df.copy()
        self.station_coords = station_coords
        self.shift_distribution = shift_distribution
        self.current_time = 0
        self.officers = { key: [] for key in shift_distribution["Early"].keys() } # Station_1, Station_2 and Station_3
        self.verbose = verbose

        # This resolved incidents is made to keep a track of when an incident
        # is allocated to an officer. This is to conduct the evaluating model check
        self.resolved_incidents = [] # This list keeps a track and order of the resolved incidents as they are allocated
    
        
        # Making the df compatible with data structures
        df.columns = [x.lower() for x in df.columns]
        

    def print_dashboard(self, allocations):
        print(f"\n--- Day {self.current_time // 24 + 1}, Hour {self.current_time % 24} ---")
        pending_cases = [inc for inc in self.cumulative_incidents if not inc.resolved]
        print(f"Pending Cases: {len(pending_cases)}")
        print("Pending Incident URNs:", [inc.urn for inc in pending_cases])

        inv_allocations = {v: k for k, v in allocations.items()}
        
        for station, officers in self.officers.items():
            print(f"\n{station}:")
            for officer in officers:
                # Find the incident the officer is currently assigned to
                if officer.name in inv_allocations.keys():
                    status = inv_allocations[officer.name]
                else: 
                    status = "Busy"
                print(f"  - {officer.name}: {status}")
        
    def calculate_distance(self, lat1, lon1, lat2, lon2):
        """
        Calculate the great circle distance between two points 
        on the earth (specified in decimal degrees)
        """
        # Convert decimal degrees to radians 
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

        # Haversine formula 
        dlat = lat2 - lat1 
        dlon = lon2 - lon1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        r = 3956  # Radius of Earth in miles. Use 6371 for kilometers
        return c * r

    def generate_incidents_for_hour(self, global_time):
        hour_incidents = self.df[(self.df['time'] == global_time)]
        incidents = []
        for _, row in hour_incidents.iterrows():
            deployment_time = row.pop("deployment time (hrs)")
            row.pop("time")  # Not needed as we use global_time
            distances = {f'Station_{i+1}': self.calculate_distance(row['latitude'], row['longitude'], lat, lon) 
                         for i, (lat, lon) in enumerate(self.station_coords)}
            incidents.append(Incident(**row, 
                                      deployment_time=deployment_time, 
                                      distances=distances,
                                      global_time=global_time))
        return incidents


    def update_officers_for_shift(self, shift):
        for station, num_officers in self.shift_distribution[shift].items():
            # Update the number of officers for each station
            self.officers[station] = [Officer(f"Officer_{station}_{shift}_{i}", station=station) for i in range(num_officers)]

    def process_allocations(self, allocations):
        if self.verbose > 0:
            print(f"{allocations=}")
        for urn, officer_id in allocations.items():
            incident = next((inc for inc in self.cumulative_incidents if inc.urn == urn), None)
            #print("incident:", incident)
            if officer_id is None:
                if incident:
                    incident.resolved = False
                continue

            officer = next((off for station_officers in self.officers.values() for off in station_officers if off.name == officer_id), None)
            if incident and officer:
                travel_time = incident.distances[officer.station] / self.SPEED_MPH
                officer.available = False  # Mark officer as busy
                officer.return_time = self.current_time + travel_time + incident.deployment_time
                
                incident.resolved = True  # Mark incident as resolved
                incident.resolving_officer = officer.name  # Assign officer to incident
                incident.response_time = self.current_time + travel_time # Global time when the response reached
                incident.resolution_time = officer.return_time # Global Time when the incident was resolved
                self.resolved_incidents.append(incident)

                
    def update_officer_availability(self):
        for station, officers in self.officers.items():
            for officer in officers:
                if not officer.available and self.current_time >= officer.return_time:
                    officer.available = True


    def run(self, model):
        self.cumulative_incidents = []  # Global list to track all incidents

        while self.current_time < 24 * 7:  # For a week-long simulation
            
            day = self.current_time // 24 + 1
            hour = self.current_time % 24
            
            # Update officer availability at the start of each timestep
            self.update_officer_availability()

            # Update officers for shift change
            if hour in [0, 8, 16]:
                shift = 'Early' if hour == 0 else 'Day' if hour == 8 else 'Night'
                #print(f"{shift=}")
                self.update_officers_for_shift(shift)
            
            total_officers = len(self.officers["Station_1"]) + len(self.officers["Station_2"]) + len(self.officers["Station_3"]) 
            #print(f"{total_officers=}")
                
            # Generate and add new incidents
            new_incidents = self.generate_incidents_for_hour(self.current_time)
            self.cumulative_incidents.extend(new_incidents)
            
            # Filter to get only pending incidents
            pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
            #print(f"{len(pending_incidents)=}")
            #print(f"{pending_incidents=}")
            # Call the model to allocate incidents
            #print(f"{self.officers=}")
            allocations = model.make_allocation(pending_incidents, self.officers, self.current_time)
            
            # Process allocations and update the state
            self.process_allocations(allocations)

            if self.verbose > 2:
                self.print_dashboard(allocations)
            
            
            # Pending cases after allocation
            pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
            #print(f"{pending_incidents=}")
            
            self.current_time += 1  # Move to the next hour
            if self.verbose > 3:
                input("Press Enter to continue to the next hour...\n")
            #print()
            #print([inc.urn for inc in self.cumulative_incidents])
            #print("*"*50)

    # Checks and Analysis
    def analyze_simulation_results(self):

        simulation = self
        
        # Initialize counters and accumulators
        incident_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        resolved_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        within_threshold_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        response_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        resolution_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        deployment_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        thresholds = {'Immediate': 1, 'Prompt': 3, 'Standard': 6}
        total_officer_hours = {officer.name: 0 for station in simulation.officers.values() for officer in station}
        unresolved_incidents = 0
    
        # Analyze each incident
        for incident in simulation.cumulative_incidents:
            priority = incident.priority
            incident_counts[priority] += 1
    
            if incident.resolved:
                resolved_counts[priority] += 1
                
                response_time = incident.response_time
                resolution_time = incident.resolution_time
                
                response_times[priority].append(response_time)
                resolution_times[priority].append(resolution_time)
                
                deployment_time = incident.deployment_time
                deployment_times[priority].append(deployment_time)
                
                # Calculate the time from incident report to response arrival
                time_to_response = incident.response_time - incident.global_time
    
    
                # Check if the response was within the threshold
                if time_to_response <= thresholds[priority]:
                    within_threshold_counts[priority] += 1
            else:
                unresolved_incidents += 1
    
        # Calculate percentages and mean times
        completion_percentages = {p: (resolved_counts[p] / incident_counts[p]) * 100 if incident_counts[p] > 0 else 0 for p in incident_counts}
        mean_response_times = {p: np.mean(response_times[p]) if response_times[p] else 0 for p in response_times}
        mean_deployment_times = {p: np.mean(deployment_times[p]) if deployment_times[p] else 0 for p in deployment_times}
        threshold_compliance = {p: (within_threshold_counts[p] / resolved_counts[p]) * 100 if resolved_counts[p] > 0 else 0 for p in incident_counts}
    
        # Calculate officer utilization
        for station in simulation.officers.values():
            for officer in station:
                if not officer.available:
                    total_officer_hours[officer.name] += (simulation.current_time - officer.return_time)
        officer_utilization = sum(total_officer_hours.values()) / (len(total_officer_hours) * simulation.current_time) * 100
    
        unresolved_incident_percentage = (unresolved_incidents / len(simulation.cumulative_incidents)) * 100 if simulation.cumulative_incidents else 0
    
        return {
            "Completion Percentages": completion_percentages,
            "Mean Response Times": mean_response_times,
            "Mean Deployment Times": mean_deployment_times,
            "Threshold Compliance": threshold_compliance,
            "Officer Utilization": officer_utilization,
            "Unresolved Incident Percentage": unresolved_incident_percentage
        }

    def check_simulation(self):
        simulation = self
        # Initialize officer assignments based on all shifts
        officer_assignments = {}
        for shift, stations in shift_distribution.items():
            for station, num_officers in stations.items():
                for i in range(num_officers):
                    officer_name = f"Officer_{station}_{shift}_{i}"
                    officer_assignments[officer_name] = []
        
        incident_response = {'Immediate': {'total': 0, 'within_time': 0}, 
                             'Prompt': {'total': 0, 'within_time': 0}, 
                             'Standard': {'total': 0, 'within_time': 0}}
        
        time_travel_occurred = False
    
        for incident in simulation.resolved_incidents:
            if incident.resolved:
                # Check officer assignments and time traveling
                if incident.resolving_officer:
                    officer_assignments[incident.resolving_officer].append(incident.resolution_time)
                    if len(officer_assignments[incident.resolving_officer]) > 1:
                        if officer_assignments[incident.resolving_officer][-2] > incident.resolution_time:
                            #print(f"{officer_assignments[incident.resolving_officer]=}")
                            #print(f"{incident.resolving_officer=}")
                            time_travel_occurred = True
    
                # Count incidents and check response time
                incident_response[incident.priority]['total'] += 1
                target_time = {'Immediate': 1, 'Prompt': 3, 'Standard': 6}[incident.priority]
                if incident.response_time - incident.global_time <= target_time:
                    incident_response[incident.priority]['within_time'] += 1
    
        # Calculate percentages
        for priority in incident_response:
            total = incident_response[priority]['total']
            if total > 0:
                incident_response[priority]['percentage'] = (incident_response[priority]['within_time'] / total) * 100
            else:
                incident_response[priority]['percentage'] = 0
    
        return officer_assignments, incident_response, time_travel_occurred

In [24]:
shift_distribution = {
    'Early': {'Station_1': 5, 'Station_2': 5, 'Station_3': 5},
    'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
    'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}
}

ps_coords = [ (p.x, p.y) for p in 
                [police_stations.one, 
                 police_stations.two, 
                 police_stations.three]]
num_officers_per_station = [4, 5, 6]
simulation = Simulation(data, ps_coords, shift_distribution, 
                        verbose=0)

In [25]:
enhanced_model = EnhancedModel()
simulation.run(enhanced_model)

In [26]:
officer_assignments, incident_response, time_travel_occurred = simulation.check_simulation()
if time_travel_occurred:
    print("ERROR: Time Travel Occured")

In [27]:
analysis = simulation.analyze_simulation_results()
analysis

{'Completion Percentages': {'Immediate': 100.0,
  'Prompt': 100.0,
  'Standard': 95.41984732824427},
 'Mean Response Times': {'Immediate': 88.30767148586473,
  'Prompt': 84.95470565734857,
  'Standard': 92.99411243046863},
 'Mean Deployment Times': {'Immediate': 1.5403225806451613,
  'Prompt': 1.5061196105702366,
  'Standard': 1.477},
 'Threshold Compliance': {'Immediate': 100.0,
  'Prompt': 92.48956884561892,
  'Standard': 64.0},
 'Officer Utilization': -0.0591858238236677,
 'Unresolved Incident Percentage': 1.056338028169014}

## Changed Simulation

In [77]:
from math import radians, cos, sin, asin, sqrt

class Simulation:
    SPEED_MPH = 30  # Speed in miles per hour

    def __init__(self, df: pd.DataFrame, station_coords: List[tuple], shift_distribution: ShiftDistribution, verbose:int=0):
        self.df = df.copy()
        self.station_coords = station_coords
        self.shift_distribution = shift_distribution
        self.current_time = 0
        self.officers = { key: [] for key in shift_distribution["Early"].keys() } # Station_1, Station_2 and Station_3
        self.verbose = verbose

        # This resolved incidents is made to keep a track of when an incident
        # is allocated to an officer. This is to conduct the evaluating model check
        self.resolved_incidents = [] # This list keeps a track and order of the resolved incidents as they are allocated
    
        
        # Making the df compatible with data structures
        df.columns = [x.lower() for x in df.columns]
        

    def print_dashboard(self, allocations):
        print(f"\n--- Day {self.current_time // 24 + 1}, Hour {self.current_time % 24} ---")
        pending_cases = [inc for inc in self.cumulative_incidents if not inc.resolved]
        print(f"Pending Cases: {len(pending_cases)}")
        print("Pending Incident URNs:", [inc.urn for inc in pending_cases])

        inv_allocations = {v: k for k, v in allocations.items()}
        
        for station, officers in self.officers.items():
            print(f"\n{station}:")
            for officer in officers:
                # Find the incident the officer is currently assigned to
                if officer.name in inv_allocations.keys():
                    status = inv_allocations[officer.name]
                else: 
                    status = "Busy"
                print(f"  - {officer.name}: {status}")
        
    def calculate_distance(self, lat1, lon1, lat2, lon2):
        """
        Calculate the great circle distance between two points 
        on the earth (specified in decimal degrees)
        """
        # Convert decimal degrees to radians 
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

        # Haversine formula 
        dlat = lat2 - lat1 
        dlon = lon2 - lon1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        r = 3956  # Radius of Earth in miles. Use 6371 for kilometers
        return c * r

    def generate_incidents_for_hour(self, global_time):
        hour_incidents = self.df[(self.df['time'] == global_time)]
        incidents = []
        for _, row in hour_incidents.iterrows():
            deployment_time = row.pop("deployment time (hrs)")
            row.pop("time")  # Not needed as we use global_time
            distances = {f'Station_{i+1}': self.calculate_distance(row['latitude'], row['longitude'], lat, lon) 
                         for i, (lat, lon) in enumerate(self.station_coords)}
            incidents.append(Incident(**row, 
                                      deployment_time=deployment_time, 
                                      distances=distances,
                                      global_time=global_time))
        return incidents


    def update_officers_for_shift(self, shift):
        new_officers = {}
        for station, num_officers in self.shift_distribution[shift].items():
            new_officers[station] = [off for off in self.officers[station] if not off.available]  # Retain busy officers
            for i in range(len(new_officers[station]), num_officers):
                new_officers[station].append(Officer(f"Officer_{station}_{shift}_{i}", station=station))
        self.officers = new_officers


    def process_allocations(self, allocations):
        if self.verbose > 0:
            print(f"{allocations=}")
        for urn, officer_id in allocations.items():
            incident = next((inc for inc in self.cumulative_incidents if inc.urn == urn), None)
            #print("incident:", incident)
            if officer_id is None:
                if incident:
                    incident.resolved = False
                continue

            officer = next((off for station_officers in self.officers.values() for off in station_officers if off.name == officer_id), None)
            if incident and officer:
                travel_time = incident.distances[officer.station] / self.SPEED_MPH
                officer.available = False  # Mark officer as busy
                officer.return_time = self.current_time + travel_time + incident.deployment_time
                
                incident.resolved = True  # Mark incident as resolved
                incident.resolving_officer = officer.name  # Assign officer to incident
                incident.response_time = self.current_time + travel_time # Global time when the response reached
                incident.resolution_time = officer.return_time # Global Time when the incident was resolved
                self.resolved_incidents.append(incident)


                
    def update_officer_availability(self):
        for station, officers in self.officers.items():
            for officer in officers:
                # Check if the officer is currently busy and if the current time is equal to or greater than the officer's return time
                if not officer.available and self.current_time >= officer.return_time:
                    # If the officer's return time has been reached or passed, make them available
                    officer.available = True


    def run(self, model):
        self.cumulative_incidents = []  # Global list to track all incidents

        while self.current_time < 24 * 7:  # For a week-long simulation
            
            day = self.current_time // 24 + 1
            hour = self.current_time % 24
            
            # Update officer availability at the start of each timestep
            self.update_officer_availability()

            # Update officers for shift change
            if hour in [0, 8, 16]:
                shift = 'Early' if hour == 0 else 'Day' if hour == 8 else 'Night'
                #print(f"{shift=}")
                self.update_officers_for_shift(shift)
            
            total_officers = len(self.officers["Station_1"]) + len(self.officers["Station_2"]) + len(self.officers["Station_3"]) 
            print(f"{total_officers=}")
                
            # Generate and add new incidents
            new_incidents = self.generate_incidents_for_hour(self.current_time)
            self.cumulative_incidents.extend(new_incidents)
            
            # Filter to get only pending incidents
            pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
            #print(f"{len(pending_incidents)=}")
            #print(f"{pending_incidents=}")
            # Call the model to allocate incidents
            #print(f"{self.officers=}")
            allocations = model.make_allocation(pending_incidents, self.officers, self.current_time)
            
            # Process allocations and update the state
            self.process_allocations(allocations)
            

            if self.verbose > 2:
                self.print_dashboard(allocations)
            
            
            # Pending cases after allocation
            pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
            #print(f"{pending_incidents=}")
            
            self.current_time += 1  # Move to the next hour
            if self.verbose > 3:
                input("Press Enter to continue to the next hour...\n")
            #print()
            #print([inc.urn for inc in self.cumulative_incidents])
            #print("*"*50)

    # Checks and Analysis
    def analyze_simulation_results(self):

        simulation = self
        
        # Initialize counters and accumulators
        incident_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        resolved_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        within_threshold_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        response_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        resolution_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        deployment_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        thresholds = {'Immediate': 1, 'Prompt': 3, 'Standard': 6}
        total_officer_hours = {officer.name: 0 for station in simulation.officers.values() for officer in station}
        unresolved_incidents = 0
    
        # Analyze each incident
        for incident in simulation.cumulative_incidents:
            priority = incident.priority
            incident_counts[priority] += 1
    
            if incident.resolved:
                resolved_counts[priority] += 1
                
                response_time = incident.response_time
                resolution_time = incident.resolution_time
                
                response_times[priority].append(response_time)
                resolution_times[priority].append(resolution_time)
                
                deployment_time = incident.deployment_time
                deployment_times[priority].append(deployment_time)
                
                # Calculate the time from incident report to response arrival
                time_to_response = incident.response_time - incident.global_time
    
    
                # Check if the response was within the threshold
                if time_to_response <= thresholds[priority]:
                    within_threshold_counts[priority] += 1
            else:
                unresolved_incidents += 1
    
        # Calculate percentages and mean times
        completion_percentages = {p: (resolved_counts[p] / incident_counts[p]) * 100 if incident_counts[p] > 0 else 0 for p in incident_counts}
        mean_response_times = {p: np.mean(response_times[p]) if response_times[p] else 0 for p in response_times}
        mean_deployment_times = {p: np.mean(deployment_times[p]) if deployment_times[p] else 0 for p in deployment_times}
        threshold_compliance = {p: (within_threshold_counts[p] / resolved_counts[p]) * 100 if resolved_counts[p] > 0 else 0 for p in incident_counts}
    
        # Calculate officer utilization
        for station in simulation.officers.values():
            for officer in station:
                if not officer.available:
                    total_officer_hours[officer.name] += (simulation.current_time - officer.return_time)
        officer_utilization = sum(total_officer_hours.values()) / (len(total_officer_hours) * simulation.current_time) * 100
    
        unresolved_incident_percentage = (unresolved_incidents / len(simulation.cumulative_incidents)) * 100 if simulation.cumulative_incidents else 0
    
        return {
            "Completion Percentages": completion_percentages,
            "Mean Response Times": mean_response_times,
            "Mean Deployment Times": mean_deployment_times,
            "Threshold Compliance": threshold_compliance,
            "Officer Utilization": officer_utilization,
            "Unresolved Incident Percentage": unresolved_incident_percentage
        }

    def check_simulation(self):
        simulation = self
        # Initialize officer assignments based on all shifts
        officer_assignments = {}
        for shift, stations in shift_distribution.items():
            for station, num_officers in stations.items():
                for i in range(num_officers):
                    officer_name = f"Officer_{station}_{shift}_{i}"
                    officer_assignments[officer_name] = []
        
        incident_response = {'Immediate': {'total': 0, 'within_time': 0}, 
                             'Prompt': {'total': 0, 'within_time': 0}, 
                             'Standard': {'total': 0, 'within_time': 0}}
        
        time_travel_occurred = False
    
        for incident in simulation.resolved_incidents:
            if incident.resolved:
                # Check officer assignments and time traveling
                if incident.resolving_officer:
                    officer_assignments[incident.resolving_officer].append(incident.resolution_time)
                    if len(officer_assignments[incident.resolving_officer]) > 1:
                        if officer_assignments[incident.resolving_officer][-2] > incident.resolution_time:
                            #print(f"{officer_assignments[incident.resolving_officer]=}")
                            #print(f"{incident.resolving_officer=}")
                            time_travel_occurred = True
    
                # Count incidents and check response time
                incident_response[incident.priority]['total'] += 1
                target_time = {'Immediate': 1, 'Prompt': 3, 'Standard': 6}[incident.priority]
                if incident.response_time - incident.global_time <= target_time:
                    incident_response[incident.priority]['within_time'] += 1
    
        # Calculate percentages
        for priority in incident_response:
            total = incident_response[priority]['total']
            if total > 0:
                incident_response[priority]['percentage'] = (incident_response[priority]['within_time'] / total) * 100
            else:
                incident_response[priority]['percentage'] = 0
    
        return officer_assignments, incident_response, time_travel_occurred

In [78]:
shift_distribution = {
    'Early': {'Station_1': 5, 'Station_2': 5, 'Station_3': 5},
    'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
    'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}
}

ps_coords = [ (p.x, p.y) for p in 
                [police_stations.one, 
                 police_stations.two, 
                 police_stations.three]]
num_officers_per_station = [4, 5, 6]
simulation = Simulation(data, ps_coords, shift_distribution, 
                        verbose=0)

In [79]:
enhanced_model = EnhancedModel()
simulation.run(enhanced_model)

total_officers=15
total_officers=15
total_officers=15
total_officers=15
total_officers=15
total_officers=15
total_officers=15
total_officers=15
total_officers=25
total_officers=25
total_officers=25
total_officers=25
total_officers=25
total_officers=25
total_officers=25
total_officers=25
total_officers=40
total_officers=40
total_officers=40
total_officers=40
total_officers=40
total_officers=40
total_officers=40
total_officers=40
total_officers=23
total_officers=23
total_officers=23
total_officers=23
total_officers=23
total_officers=23
total_officers=23
total_officers=23
total_officers=25
total_officers=25
total_officers=25
total_officers=25
total_officers=25
total_officers=25
total_officers=25
total_officers=25
total_officers=40
total_officers=40
total_officers=40
total_officers=40
total_officers=40
total_officers=40
total_officers=40
total_officers=40
total_officers=18
total_officers=18
total_officers=18
total_officers=18
total_officers=18
total_officers=18
total_officers=18
total_offi

In [75]:
officer_assignments, incident_response, time_travel_occurred = simulation.check_simulation()
if time_travel_occurred:
    print("ERROR: Time Travel Occured")

ERROR: Time Travel Occured


In [76]:
analysis = simulation.analyze_simulation_results()
analysis

{'Completion Percentages': {'Immediate': 100.0,
  'Prompt': 100.0,
  'Standard': 94.46564885496184},
 'Mean Response Times': {'Immediate': 88.33909852120367,
  'Prompt': 84.87578981949687,
  'Standard': 92.61112158434575},
 'Mean Deployment Times': {'Immediate': 1.5403225806451613,
  'Prompt': 1.5061196105702366,
  'Standard': 1.4758585858585858},
 'Threshold Compliance': {'Immediate': 96.7741935483871,
  'Prompt': 93.88038942976355,
  'Standard': 66.66666666666666},
 'Officer Utilization': -0.11971319023174223,
 'Unresolved Incident Percentage': 1.2764084507042255}

### Simulation with max officer utilisation

In [7]:
from math import radians, cos, sin, asin, sqrt

class SimulationWithMaxUtilisation:
    SPEED_MPH = 30  # Speed in miles per hour

    def __init__(self, df: pd.DataFrame, station_coords: List[tuple], shift_distribution: ShiftDistribution, verbose:int=0):
        self.df = df.copy()
        self.station_coords = station_coords
        self.shift_distribution = shift_distribution
        self.current_time = 0
        self.officers = { key: [] for key in shift_distribution["Early"].keys() } # Station_1, Station_2 and Station_3
        self.verbose = verbose
        self.hours = [i for i in range(24*7 + 1)]
        self.hour_index = 0
        self.return_times = [] # array keeps a track of return times of officers

        # This resolved incidents is made to keep a track of when an incident
        # is allocated to an officer. This is to conduct the evaluating model check
        self.resolved_incidents = [] # This list keeps a track and order of the resolved incidents as they are allocated
    
        
        # Making the df compatible with data structures
        df.columns = [x.lower() for x in df.columns]
        

    def print_dashboard(self, allocations):
        print(f"\n--- Day {self.current_time // 24 + 1}, Hour {self.current_time % 24} ---")
        pending_cases = [inc for inc in self.cumulative_incidents if not inc.resolved]
        print(f"Pending Cases: {len(pending_cases)}")
        print("Pending Incident URNs:", [inc.urn for inc in pending_cases])

        inv_allocations = {v: k for k, v in allocations.items()}
        
        for station, officers in self.officers.items():
            print(f"\n{station}:")
            for officer in officers:
                # Find the incident the officer is currently assigned to
                if officer.name in inv_allocations.keys():
                    status = inv_allocations[officer.name]
                else: 
                    status = "Busy"
                print(f"  - {officer.name}: {status}")
        
    def calculate_distance(self, lat1, lon1, lat2, lon2):
        """
        Calculate the great circle distance between two points 
        on the earth (specified in decimal degrees)
        """
        # Convert decimal degrees to radians 
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

        # Haversine formula 
        dlat = lat2 - lat1 
        dlon = lon2 - lon1 
        a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
        c = 2 * asin(sqrt(a)) 
        r = 3956  # Radius of Earth in miles. Use 6371 for kilometers
        return c * r

    def generate_incidents_for_hour(self, global_time):
        hour_incidents = self.df[(self.df['time'] == global_time)]
        incidents = []
        for _, row in hour_incidents.iterrows():
            deployment_time = row.pop("deployment time (hrs)")
            row.pop("time")  # Not needed as we use global_time
            distances = {f'Station_{i+1}': self.calculate_distance(row['latitude'], row['longitude'], lat, lon) 
                         for i, (lat, lon) in enumerate(self.station_coords)}
            incidents.append(Incident(**row, 
                                      deployment_time=deployment_time, 
                                      distances=distances,
                                      global_time=global_time))
        return incidents


    def update_officers_for_shift(self, shift):
        for station, num_officers in self.shift_distribution[shift].items():
            # Update the number of officers for each station
            self.officers[station] = [Officer(f"Officer_{station}_{shift}_{i}", station=station) for i in range(num_officers)]

    def process_allocations(self, allocations):
        if self.verbose > 0:
            print(f"{allocations=}")
        for urn, officer_id in allocations.items():
            incident = next((inc for inc in self.cumulative_incidents if inc.urn == urn), None)
            #print("incident:", incident)
            if officer_id is None:
                if incident:
                    incident.resolved = False
                continue

            officer = next((off for station_officers in self.officers.values() for off in station_officers if off.name == officer_id), None)
            if incident and officer:
                travel_time = incident.distances[officer.station] / self.SPEED_MPH
                officer.available = False  # Mark officer as busy
                officer.return_time = self.current_time + travel_time + incident.deployment_time
                
                self.return_times.append(officer.return_time)
            
                incident.resolved = True  # Mark incident as resolved
                incident.resolving_officer = officer.name  # Assign officer to incident
                incident.response_time = self.current_time + travel_time # Global time when the response reached
                incident.resolution_time = officer.return_time # Global Time when the incident was resolved
                self.resolved_incidents.append(incident)
                

                
    def update_officer_availability(self):
        for station, officers in self.officers.items():
            for officer in officers:
                if not officer.available and self.current_time >= officer.return_time:
                    officer.available = True

    
    def get_return_times_for_next_hour(self):
        global_time = self.current_time
        return_times_for_next_hour = [rt for rt in self.return_times if rt <= global_time + 1]
        return sorted(return_times_for_next_hour)
                    
                    
    def run(self, model):
        self.cumulative_incidents = []  # Global list to track all incidents

        while self.current_time < 24 * 7:  # For a week-long simulation
            
            day = self.current_time // 24 + 1
            hour = self.current_time % 24
            
            # Update officer availability at the start of each timestep
            self.update_officer_availability()

            # Update officers for shift change
            if hour in [0, 8, 16]:
                shift = 'Early' if hour == 0 else 'Day' if hour == 8 else 'Night'
                #print(f"{shift=}")
                self.update_officers_for_shift(shift)
            
            total_officers = len(self.officers["Station_1"]) + len(self.officers["Station_2"]) + len(self.officers["Station_3"]) 
            #print(f"{total_officers=}")
                
            # Generate and add new incidents
            new_incidents = self.generate_incidents_for_hour(self.current_time)
            self.cumulative_incidents.extend(new_incidents)
            
            # Filter to get only pending incidents
            pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
            #print(f"{len(pending_incidents)=}")
            #print(f"{pending_incidents=}")
            # Call the model to allocate incidents
            #print(f"{self.officers=}")
            allocations = model.make_allocation(pending_incidents, self.officers, self.current_time)
            
            # Process allocations and update the state
            self.process_allocations(allocations)
            
            # After making the allocations for the hour
            # get to each return time of the officer
            # and make new allocations
            return_times_within_hour = self.get_return_times_for_next_hour()
            #print(f"{self.current_time=} {return_times_within_hour=}")
            
            for time in return_times_within_hour:
                self.current_time = time
                
                self.update_officer_availability()
                pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
                allocations = model.make_allocation(pending_incidents, self.officers, self.current_time)
                # Process allocations and update the state
                self.process_allocations(allocations)
                self.return_times.remove(time)
                
                

            if self.verbose > 2:
                self.print_dashboard(allocations)
            
            
            # Pending cases after allocation
            pending_incidents = [inc for inc in self.cumulative_incidents if not inc.resolved]
            #print(f"{pending_incidents=}")
            
            #self.current_time += 1  # Move to the next hour
            self.hour_index += 1
            self.current_time = self.hours[self.hour_index]
            
            if self.verbose > 3:
                input("Press Enter to continue to the next hour...\n")
            #print()
            #print([inc.urn for inc in self.cumulative_incidents])
            #print("*"*50)

    # Checks and Analysis
    def analyze_simulation_results(self):

        simulation = self
        
        # Initialize counters and accumulators
        incident_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        resolved_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        within_threshold_counts = {'Immediate': 0, 'Prompt': 0, 'Standard': 0}
        response_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        resolution_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        deployment_times = {'Immediate': [], 'Prompt': [], 'Standard': []}
        thresholds = {'Immediate': 1, 'Prompt': 3, 'Standard': 6}
        total_officer_hours = {}
        unresolved_incidents = 0
    
        # Analyze each incident
        for incident in simulation.cumulative_incidents:
            priority = incident.priority
            incident_counts[priority] += 1
    
            if incident.resolved:
                resolved_counts[priority] += 1
                
                response_time = incident.response_time
                resolution_time = incident.resolution_time
                
                response_times[priority].append(response_time)
                resolution_times[priority].append(resolution_time)
                
                deployment_time = incident.deployment_time
                deployment_times[priority].append(deployment_time)
                
                if incident.resolving_officer not in total_officer_hours.keys():
                    total_officer_hours[incident.resolving_officer]  = 0
                
                time_spent_on_incident = incident.resolution_time - incident.global_time
                total_officer_hours[incident.resolving_officer] += time_spent_on_incident
                
                # Calculate the time from incident report to response arrival
                time_to_response = incident.response_time - incident.global_time
    
    
                # Check if the response was within the threshold
                if time_to_response <= thresholds[priority]:
                    within_threshold_counts[priority] += 1
            else:
                unresolved_incidents += 1
    
        # Calculate percentages and mean times
        completion_percentages = {p: (resolved_counts[p] / incident_counts[p]) * 100 if incident_counts[p] > 0 else 0 for p in incident_counts}
        mean_response_times = {p: np.mean(response_times[p]) if response_times[p] else 0 for p in response_times}
        mean_deployment_times = {p: np.mean(deployment_times[p]) if deployment_times[p] else 0 for p in deployment_times}
        threshold_compliance = {p: (within_threshold_counts[p] / resolved_counts[p]) * 100 if resolved_counts[p] > 0 else 0 for p in incident_counts}
    
        # Calculate officer utilization
        #for station in simulation.officers.values():
        #    for officer in station:
        #        if not officer.available:
        #            total_officer_hours[officer.name] += (simulation.current_time - officer.return_time)
        
        #officer_utilization = sum(total_officer_hours.values()) / (len(total_officer_hours) * simulation.current_time) * 100
        sum_total_office_hours = sum(total_officer_hours.values())
        unresolved_incident_percentage = (unresolved_incidents / len(simulation.cumulative_incidents)) * 100 if simulation.cumulative_incidents else 0
    
        return {
            "Completion Percentages": completion_percentages,
            "Mean Response Times": mean_response_times,
            "Mean Deployment Times": mean_deployment_times,
            "Threshold Compliance": threshold_compliance,
            #"Officer Utilization": officer_utilization,
            "Total Officer Hours": sum_total_office_hours, 
            "Unresolved Incident Percentage": unresolved_incident_percentage
        }

    def check_simulation(self):
        simulation = self
        # Initialize officer assignments based on all shifts
        officer_assignments = {}
        for shift, stations in shift_distribution.items():
            for station, num_officers in stations.items():
                for i in range(num_officers):
                    officer_name = f"Officer_{station}_{shift}_{i}"
                    officer_assignments[officer_name] = []
        
        incident_response = {'Immediate': {'total': 0, 'within_time': 0}, 
                             'Prompt': {'total': 0, 'within_time': 0}, 
                             'Standard': {'total': 0, 'within_time': 0}}
        
        time_travel_occurred = False
    
        for incident in simulation.resolved_incidents:
            if incident.resolved:
                # Check officer assignments and time traveling
                if incident.resolving_officer:
                    officer_assignments[incident.resolving_officer].append(incident.resolution_time)
                    if len(officer_assignments[incident.resolving_officer]) > 1:
                        if officer_assignments[incident.resolving_officer][-2] > incident.resolution_time:
                            time_travel_occurred = True
    
                # Count incidents and check response time
                incident_response[incident.priority]['total'] += 1
                target_time = {'Immediate': 1, 'Prompt': 3, 'Standard': 6}[incident.priority]
                if incident.response_time - incident.global_time <= target_time:
                    incident_response[incident.priority]['within_time'] += 1
    
        # Calculate percentages
        for priority in incident_response:
            total = incident_response[priority]['total']
            if total > 0:
                incident_response[priority]['percentage'] = (incident_response[priority]['within_time'] / total) * 100
            else:
                incident_response[priority]['percentage'] = 0
    
        return officer_assignments, incident_response, time_travel_occurred

In [11]:
shift_distribution = {
    'Early': {'Station_1': 5, 'Station_2': 5, 'Station_3': 5},
    'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
    'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}
}

ps_coords = [ (p.x, p.y) for p in 
                [police_stations.one, 
                 police_stations.two, 
                 police_stations.three]]

simulation = SimulationWithMaxUtilisation(data, ps_coords, shift_distribution, 
                        verbose=0)
naive_model = NaiveModel()
simulation.run(naive_model)

In [12]:
officer_assignments, incident_response, time_travel_occurred = simulation.check_simulation()
if time_travel_occurred:
    print("ERROR: Time Travel Occured")

In [13]:
simulation.analyze_simulation_results()

{'Completion Percentages': {'Immediate': 100.0,
  'Prompt': 100.0,
  'Standard': 100.0},
 'Mean Response Times': {'Immediate': 88.33043348830911,
  'Prompt': 84.65961030075212,
  'Standard': 88.37365649837112},
 'Mean Deployment Times': {'Immediate': 1.5403225806451613,
  'Prompt': 1.5061196105702366,
  'Standard': 1.4796755725190842},
 'Threshold Compliance': {'Immediate': 100.0,
  'Prompt': 98.95688456189151,
  'Standard': 98.09160305343512},
 'Total Officer Hours': 4422.399999003844,
 'Unresolved Incident Percentage': 0.0}

In [14]:
shift_distribution = {
    'Early': {'Station_1': 5, 'Station_2': 5, 'Station_3': 5},
    'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
    'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}
}

ps_coords = [ (p.x, p.y) for p in 
                [police_stations.one, 
                 police_stations.two, 
                 police_stations.three]]

simulation = SimulationWithMaxUtilisation(data, ps_coords, shift_distribution, 
                        verbose=0)
enhanced_model = EnhancedModel()
simulation.run(enhanced_model)

NameError: name 'EnhancedModel' is not defined

In [15]:
officer_assignments, incident_response, time_travel_occurred = simulation.check_simulation()
if time_travel_occurred:
    print("ERROR: Time Travel Occured")

In [16]:
simulation.analyze_simulation_results()

AttributeError: 'SimulationWithMaxUtilisation' object has no attribute 'cumulative_incidents'

# Attempt at reinforcement learning

In [18]:
class RLEnvironmentModified:
    """
    Custom RL environment that interacts with the user's simulation.
    """
    def __init__(self, simulation):
        """
        Initialize the RL environment with the user's simulation.
        :param simulation: Instance of the SimulationWithMaxUtilisation class.
        """
        self.simulation = simulation
        self.state = self.get_current_state()
        self.action_space = self.define_action_space()

    def get_current_state(self):
        """
        Retrieve the current state from the simulation.
        This method should extract relevant state information from the simulation class.
        """
        # Extracting state information from the simulation
        return {
            "current_time": self.simulation.current_time,
            "officers": self.simulation.officers,
            "cumulative_incidents": self.simulation.cumulative_incidents
        }

    def define_action_space(self):
        """
        Define the action space for the RL environment.
        This should correspond to the types of decisions/actions possible in the simulation.
        """
        # Defining action space based on allocating officers to incidents
        # Assuming the format of actions as (incident_id, officer_id)
        return [(incident.urn, officer.name) 
                for incident in self.simulation.cumulative_incidents 
                for officer_list in self.simulation.officers.values() 
                for officer in officer_list]

    def reset(self):
        """
        Reset the environment to an initial state.
        """
        # Reset the simulation and update the state
        self.simulation.reset()  # Assuming a reset method in the Simulation class
        self.state = self.get_current_state()
        return self.state

    def step(self, action):
        """
        Apply an action to the environment and observe the result.
        :param action: The action chosen by the RL agent.
        """
        # Apply action in the simulation and calculate the reward
        # Action format: (incident_id, officer_id)
        incident_id, officer_id = action
        allocations = {incident_id: officer_id}  # Mapping incident to officer
        self.simulation.process_allocations(allocations)

        # Update state and calculate reward
        next_state = self.get_current_state()
        reward = self.calculate_reward(allocations)  # Assuming a method to calculate reward
        done = self.simulation.is_done()  # Check if the simulation is over
        return next_state, reward, done

    def calculate_reward(self, allocations):
        """
        Calculate the reward based on the simulation's outcome.
        This method should be defined based on the goals of the simulation.
        """
        # Placeholder: Implement logic to calculate reward
        # Example: reward based on the number of successfully resolved incidents
        # and compliance with response time targets
        reward = 0
        for incident_id in allocations.keys():
            incident = next((inc for inc in self.simulation.cumulative_incidents if inc.urn == incident_id), None)
            if incident and incident.resolved:
                # Add rewards based on incident resolution and response time
                reward += 1  # Example reward logic, needs to be defined based on simulation goals
        return reward

# Placeholder: Initialize the RLEnvironment with an instance of the user's Simulation class
# simulation_instance = SimulationWithMaxUtilisation(...)  # Create an instance of the Simulation class
# rl_env = RLEnvironmentModified(simulation_instance)

# This integration aligns the RL environment with the structure and logic of the SimulationWithMaxUtilisation class.
# The reward calculation logic needs to be defined based on the specific goals and metrics of the simulation.


In [19]:
shift_distribution = {
    'Early': {'Station_1': 5, 'Station_2': 5, 'Station_3': 5},
    'Day': {'Station_1': 8, 'Station_2': 8, 'Station_3': 9},
    'Night': {'Station_1': 13, 'Station_2': 13, 'Station_3': 14}
}

ps_coords = [ (p.x, p.y) for p in 
                [police_stations.one, 
                 police_stations.two, 
                 police_stations.three]]
num_officers_per_station = [4, 5, 6]
simulation = Simulation(data, ps_coords, shift_distribution, 
                        verbose=0)

In [21]:
# Initialize the simulation and the RL environment
simulation_instance = simulation  # Replace with your simulation setup
rl_env = RLEnvironmentModified(simulation_instance)

# Initialize the RL agent
rl_agent = QLearningAgent(rl_env)

# Set up the training loop
num_episodes = 1000  # Define the number of episodes for training

for episode in range(num_episodes):
    state = rl_env.reset()
    done = False

    while not done:
        action = rl_agent.choose_action(state)
        next_state, reward, done = rl_env.step(action)
        rl_agent.learn(state, action, reward, next_state)
        state = next_state

    # Optionally: Print out rewards or other metrics to monitor progress

# After training, test the agent's performance
# (This could involve running the simulation with the trained agent and observing its decisions)


AttributeError: 'Simulation' object has no attribute 'cumulative_incidents'

# Dynamic Programming

In [37]:
def dynamic_programming_allocation(incidents, officers, current_time):
    """
    Dynamic programming approach to allocate officers to incidents based on the defined metric.
    :param incidents: List of incidents.
    :param officers: List of officers.
    :param current_time: Current time in the simulation.
    :return: Allocation of officers to incidents.
    """
    incidents.sort(key=lambda inc: (-calculate_metric(inc, officers[0], current_time), inc.global_time))

    allocations = {}
    for incident in incidents:
        best_officer = None
        best_metric = float('-inf')

        for officer in officers:
            if officer.available:
                metric = calculate_metric(incident, officer, current_time)
                if metric > best_metric:
                    best_metric = metric
                    best_officer = officer

        if best_officer:
            allocations[incident.urn] = best_officer.name
            best_officer.available = False

    return allocations


In [38]:
def calculate_metric(incident, officer, current_time):
    """
    Calculate the metric for assigning an officer to an incident.
    :param incident: The incident object.
    :param officer: The officer object.
    :param current_time: The current time in the simulation.
    :return: The calculated metric value.
    """
    priority_weight = {'Immediate': 3, 'Prompt': 2, 'Standard': 1}
    distance = incident.distances[officer.station]
    lateness = max(0, current_time - incident.global_time)  # Lateness in responding
    priority = priority_weight[incident.priority]

    # Metric: Weighted sum of priority, distance, and lateness
    metric = priority - (0.1 * distance) - (0.5 * lateness)
    return metric


In [39]:
# Initialize the simulation
simulation = SimulationWithMaxUtilisation(data, ps_coords, shift_distribution, verbose=0)

# Run the simulation for a predefined duration (e.g., one week or 168 hours)
simulation_duration = 24 * 7  # 7 days
while simulation.current_time < simulation_duration:
    current_time = simulation.current_time
    officers = [officer for station_officers in simulation.officers.values() for officer in station_officers if officer.available]
    
    # Generate incidents for the current hour
    incidents = simulation.generate_incidents_for_hour(current_time)

    # Make allocations using the dynamic programming approach
    allocations = dynamic_programming_allocation(incidents, officers, current_time)

    # Process allocations in the simulation
    simulation.process_allocations(allocations)

    # Advance the simulation to the next time step
    # Assuming the simulation has a method to move to the next hour
    simulation.advance_time()  # Ensure this method is implemented in your simulation class

# Analyze the results after the simulation
results = simulation.analyze_simulation_results()


AttributeError: 'SimulationWithMaxUtilisation' object has no attribute 'advance_time'